Begin Milestone 1 with a 250-500-word narrative describing your original idea for the analysis/model building business problem. Clearly identify the problem you will address and the target for your model. Then, do a graphical analysis creating a minimum of four graphs. Label your graphs appropriately and explain/analyze the information provided by each graph. Your analysis should begin to answer the question(s) you are addressing. Write a short overview/conclusion of the insights gained from your graphical analysis.

In [ ]:
#importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = read_csv('redfin.csv')

In [ ]:
#checking columns for data
df.head()

In [ ]:
#looking at num of rows/columns
df.shape()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#checking null values
df.isnull()